In [7]:
import pandas as pd
import numpy as np

In [8]:
from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
data = pd.read_csv('/content/drive/My Drive/Summer Research 24/NycMayorVote/aggregated_voting_NYCMayor.csv')
data

,1st,2nd,3rd,4th,5th,count
0,217572,Null,Null,Null,Null,64274
1,217796,Null,Null,Null,Null,20566
2,219469,Null,Null,Null,Null,12771
3,219978,Null,Null,Null,Null,10258
4,217572,219469,Null,Null,Null,9833
...,...,...,...,...,...,...
62149,217796,218117,219469,217605,218922,1
62150,219469,218117,221183,217605,218922,1
62151,217796,218117,219469,217605,Null,1
62152,219469,218117,221183,217605,217654,1


Copeland


In [11]:
from collections import defaultdict

def copeland_winner(df):
    candidates = set(df['1st']).union(set(df['2nd'])).union(set(df['3rd'])).union(set(df['4th'])).union(set(df['5th']))
    pairwise_wins = defaultdict(int)
    pairwise_losses = defaultdict(int)

    for candidate1 in candidates:
        for candidate2 in candidates:
            if candidate1 != candidate2:
                wins = 0
                losses = 0
                for _, row in df.iterrows():
                     # Get the ranks if the candidate is present in the row, otherwise assign a large rank (like len(candidates))
                    rank1 = row[['1st', '2nd', '3rd', '4th', '5th']].tolist().index(candidate1) if candidate1 in row[['1st', '2nd', '3rd', '4th', '5th']].tolist() else len(candidates)
                    rank2 = row[['1st', '2nd', '3rd', '4th', '5th']].tolist().index(candidate2) if candidate2 in row[['1st', '2nd', '3rd', '4th', '5th']].tolist() else len(candidates)
                    if rank1 < rank2:
                        wins += row['count']
                    elif rank1 > rank2:
                        losses += row['count']
                if wins > losses:
                    pairwise_wins[candidate1] += 1
                    pairwise_losses[candidate2] += 1
                elif losses > wins:
                    pairwise_wins[candidate2] += 1
                    pairwise_losses[candidate1] += 1

    copeland_scores = {candidate: pairwise_wins[candidate] - pairwise_losses[candidate] for candidate in candidates}
    winner = max(copeland_scores, key=copeland_scores.get)

    return winner, copeland_scores

winner, scores = copeland_winner(data)
print(f"Winner: {winner}")
print(f"Copeland Scores: {scores}")

KeyboardInterrupt: 